# Estimating Amount of Closures (Issue 19)

## Elliot Frank
Issue #19 in https://github.com/jasonasher/dc_doh_hackathon/issues requires that we know which inspections resulted in a closure. It seems that this isn't a field that's accessible in the summary data, so instead we're attempting to estimate if a given inspection resulted in a closure.

We're going to first do two estimates. First, we'll use the number of restorations as a proxy for the number of closures, because generally a restoration immediately follows a closure. Second, we'll look at each inspection to see if the word "closure" appears in the inspector's comments.

In [339]:
# First load in data
import csv
import pandas as pd

#data_16 = pd.read_csv('Basic_Business_License_in_2016.csv')
inspections_geo = pd.read_csv('inspections_geocoded.csv')
inspections_sum = pd.read_csv('potential_inspection_summary_data.csv',low_memory=False)
violation_details= pd.read_csv('potential_violation_details_data.csv')
inspections_sum = inspections_sum[inspections_sum['known_valid'] == True]
inspections_sum.loc[:,'inspection_date'] = pd.to_datetime(inspections_sum.loc[:,'inspection_date'])


# Estimate closures using "restorations" as a proxy

In [340]:
def process_inspection_type(x):
    if isinstance(x,str):
        x = x.lower()
    elif x != x:
        x = 'unknown'
    else:
        x = 'unknown'
    return x

def find_restoration(x):
    if x.find('restor') > -1:
        return True
    else:
        return False

inspections_sum.loc[:,'inspection_type'] = inspections_sum['inspection_type'].apply(process_inspection_type)

inspections_sum['restoration?'] = inspections_sum['inspection_type'].apply(find_restoration)



Look at the different types of "restorations" we found with out search.

In [341]:
restorations = inspections_sum[inspections_sum['restoration?'] == True]
print(restorations['inspection_type'].value_counts(),'\n')
print('size =',restorations['inspection_type'].size)

restoration                        758
license restoration                 47
flood restoration                    3
fire restoration                     3
restoratioin                         2
restoration from6/8/17 closure.      1
restoration/license renewal          1
restoration, ownership               1
restoraiton                          1
restoration.                         1
remodel restoration                  1
rrestoration                         1
renewal/restoration                  1
restorations                         1
Name: inspection_type, dtype: int64 

size = 822


# Estimating via Comments

First we have to process the comments, so we make a Dataframew with just comments.

In [342]:
def process_comments(x):
    if isinstance(x,str):
        x = x.lower()
    else:
        x = 'no comment'
    return x

def find_suspensions(x):
    if x.find('suspension') > -1:
        return True
    elif x.find('closure') > -1:
        return True
    else:
        return False
    
inspections_sum.loc[:,'inspector_comments'] = inspections_sum.loc[:,'inspector_comments'].apply(process_comments)
inspections_sum['suspended?'] = inspections_sum.loc[:,'inspector_comments'].apply(find_suspensions)


In [343]:
suspensions = inspections_sum[inspections_sum['suspended?'] == True]
print('size =',suspensions['inspection_type'].size)

size = 938


In [344]:
# Look at a few comments
pd.options.display.max_colwidth = 500
for row in suspensions.loc[:,'inspector_comments'][100:150]:
    print(row , '\n')

certified food manager came in before inspection was finish summary suspension in order for license to be restored, a reinspection fee of $100.00 dollar normal business hour or $400 non-business hour must be paid and all items must be corrected and approved by the doh . if you have any questions, please call area supervisor mr. ronnie taylor at 202-442-9037. 

license restored from suspension on february 2, 2012 exterminator was out to service business on 2/2/2012 smart h & a/c company was out to service hot water heater and garbage disposal. garbage disposal need a part order was placed delivery in 2 to 3 days. if you have any questions, please call area supervisor mr. ronnie taylor at 202-442-9037. 

summary suspension abated from inspection conducted on february 3, 2012. if you have any questions, please call area supervisor mr. ronnie taylor at 202-442-9037. contact dcra regarding license status. please fax renewed license to our office at 202-535-2180. 

summary suspension correct

## Some conclusions
So in our dataset we got two estimates for the amount of closures: there were 822 according to the restoration method, and 938 according to the comments. 

However, when we read the comments, we notice that occasionally the inspector uses the term "summary suspension" within the course of saying that, for example, "a summary suspension has been lifted" – which might throw off our count.

Now, to improve our estimate, we're going to try to integrate the two methods: first we'll do this in a basic way, by seeing how many addresses they have in common. Addresses aren't a perfect way to find distinct restaurants (for example, many restaurants point to a single address in L'Enfant plaza) but they're the best thing at the moment.

In [379]:
restored_addresses = list(set(restorations.address))
suspended_addresses = list(set(suspensions.address))

def intersect(a, b):
     return list(set(a) & set(b))

print('# distinct addresses in the restored estimate = ',len(restored_addresses))
print('# distinct addresses in the suspended estimate = ',len(suspended_addresses))
print('# of addresses that appear in both = ',len(intersect(restored_addresses,suspended_addresses)))


# distinct addresses in the restored estimate =  616
# distinct addresses in the suspended estimate =  666
# of addresses that appear in both =  553


We can also look at if the yearly distributions of the two data sets are similar.

In [380]:
print(restorations.inspection_date.apply(lambda x: x.year).value_counts().sort_index(),'\n')
print(suspensions.inspection_date.apply(lambda x: x.year).value_counts().sort_index(),'\n')

2010     15
2011     38
2012     52
2013     69
2014    181
2015    175
2016    171
2017    121
Name: inspection_date, dtype: int64 

2010     24
2011     54
2012     71
2013     76
2014    213
2015    193
2016    169
2017    138
Name: inspection_date, dtype: int64 



# To Do Next

As we've seen, it's challenging to try to determine if a given inspection resulted in a closure. One idea is to find pairs of inspections on the same restaurant within a few weeks of each other. Generally, if an inspector closes a restaurant they will return in about a week to determine if the restaurant has complied with their recommendations and can be reopened. Roughly, we think the inspections in "suspensions" would correspond to the shutting-down inspection, and "restorations" would correspond to the re-opening inspection. 

Unfortunately, matching inspections in this way is difficult! We don't have a single list of all the inspections for each restaurant. This is another data challenge. 